<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий на hh.ru
   

In [ ]:
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
print(plotly.__version__)

# Исследование структуры данных

#### 1. Прочитайте данные с помощью библиотеки Pandas. Совет: перед чтением обратите внимание на разделитель внутри файла. 

In [ ]:
hh_df = pd.read_csv('data/dst-3.0_16_1_hh_database.csv', sep=';')
# hh_df.head()

#### 2. Выведите несколько первых (последних) строк таблицы, чтобы убедиться в том, что ваши данные не повреждены. Ознакомьтесь с признаками и их структурой.

In [ ]:
# первые 2 строки таблицы:
hh_df.head(2)

In [ ]:
# посдедние 2 строки таблицы
hh_df.tail(2)

In [ ]:
print(f'Размерность таблицы {hh_df.shape}')
#Для анализа есть 12 многосоставных критериев (в один столбец внесены сразу несколько параметров)
#и  44744 тысяч резюме

#### Все признаки, в которых есть пропуски в данных:

In [ ]:
display(hh_df.info())

#### 3. Oсновная информация о числе непустых значений в столбцах и их типах в таблице.

In [ ]:
#ваш код здесь
hh_df_null = hh_df.isnull().sum()
hh_df_null[hh_df_null > 0]

#### 4. Обращаем внимание на информацию о числе непустых значений.

In [ ]:
hh_df_null_percent = hh_df.isnull().mean() * 100
hh_df_with_null = hh_df_null_percent[hh_df_null_percent>0].sort_values(ascending=False)
hh_df_with_null

#### Выведем общее количество пропусков в виде тепловой карты по всему датафрейму.

In [ ]:
px.imshow(hh_df.isnull().astype('int')) 

#### Наглядно видно, что все данные заполнены почти полностью. Оценили незначительность пропусков данных по столбцу "Опыт работы" относительно к общему массиву данных.

#### 5. Выведим основную статистическую информацию о столбцах.


In [ ]:
hh_df.describe()

# Преобразование данных

### 1. Начнем с простого - с признака **"Образование и ВУЗ"**. Его текущий формат это: **<Уровень образования год выпуска ВУЗ специальность...>**. Например:
* Высшее образование 2016 Московский авиационный институт (национальный исследовательский университет)...
* Неоконченное высшее образование 2000  Балтийская государственная академия рыбопромыслового флота…
#### Нас будет интересовать только уровень образования.

#### Создаём с помощью функции-преобразования новый признак **"Образование"**, который должен иметь 4 категории: "высшее", "неоконченное высшее", "среднее специальное" и "среднее".
#### Выполняем преобразование, отвечаем на контрольные вопросы и удаляем признак "Образование и ВУЗ".
* Совет: - обратите внимание на структуру текста в столбце **"Образование и ВУЗ"**. Гарантируется, что текущий уровень образования соискателя всегда находится в первых 2ух слов и начинается с заглавной буквы. Воспользуйтесь этим.
* Совет:  - проверяйте полученные категории, например, с помощью - метода unique()*


In [ ]:
#Создаем с помощью lambda функции новый признак "Образование" выделяя часть с образованием
hh_df["Образование"] = hh_df["Образование и ВУЗ"].apply(lambda x: ' '.join(x.split(' '))[:10])

#Выделяем и переименовываем 4 категории: "высшее", "неоконченное высшее", "среднее специальное" и "среднее".
hh_df.loc[(hh_df["Образование"] == 'Неоконченн', ("Образование"))] = 'неоконченное высшее'
hh_df.loc[(hh_df["Образование"] == 'Среднее сп', ("Образование"))] = 'среднее специальное'
hh_df.loc[(hh_df["Образование"] == 'Высшее обр', ("Образование"))] = 'высшее'
hh_df.loc[(hh_df["Образование"] == 'Среднее об', ("Образование"))] = 'среднее'

#Проверяем, что кроме 4 категорий нет лишних
hh_df["Образование"].unique() 

#удаляем столбец "Образование и ВУЗ"
hh_df = hh_df.drop('Образование и ВУЗ', axis=1)  


print(hh_df['Образование'].value_counts()['среднее'])

### 2. Теперь нас интересует столбец **"Пол, возраст"**. Сейчас он представлен в формате **<Пол , возраст , дата рождения >**. Например:
* Мужчина , 39 лет , родился 27 ноября 1979 
* Женщина , 21 год , родилась 13 января 2000
#### Как мы понимаем, нам необходимо выделить каждый параметр в отдельный столбец.

#### Создаём два новых признака **"Пол"** и **"Возраст"**. При этом важно учесть:
* Признак пола должен иметь 2 уникальных строковых значения: 'М' - мужчина, 'Ж' - женщина. 
* Признак возраста должен быть представлен целыми числами.
#### Выполняем преобразование, отвечаем на контрольные вопросы и удаляем признак **"Пол, возраст"** из таблицы.
*Совет: обратите внимание на структуру текста в столбце, в части на то, как разделены параметры пола, возраста и даты рождения между собой - символом ' , '. 
Гарантируется, что структура одинакова для всех строк в таблице. Вы можете воспользоваться этим.*


In [ ]:
def get_sex(arg):
    if 'Мужчина' in arg:
        return 'М'
    else:
        return 'Ж'
    
def get_age(arg):
    arg_splitted = arg.split(' ')
    year_words=['год', 'года', 'лет']
    for index, item in enumerate (arg_splitted):
        if item in year_words:
            return int(arg_splitted[index-1])

hh_df['Пол'] = hh_df['Пол, возраст'].apply(get_sex)
hh_df['Возраст'] = hh_df['Пол, возраст'].apply(get_age)
hh_df = hh_df.drop('Пол, возраст', axis=1)

print(round(hh_df['Пол'].value_counts(normalize=True)['Ж'] * 100, 2))
print(round(hh_df['Возраст'].mean(), 2))

### 3. Следующим этапом преобразуем признак **"Опыт работы"**. Его текущий формат - это: **<Опыт работы: n лет m месяцев, периоды работы в различных компаниях…>**. 

Из столбца нам необходимо выделить общий опыт работы соискателя в месяцах, новый признак назовем "Опыт работы (месяц)"

Для начала обсудим условия решения задачи:
* Во-первых, в данном признаке есть пропуски. Условимся, что если мы встречаем пропуск, оставляем его как есть (функция-преобразование возвращает NaN)
* Во-вторых, в данном признаке есть скрытые пропуски. Для некоторых соискателей в столбце стоит значения "Не указано". Их тоже обозначим как NaN (функция-преобразование возвращает NaN)
* В-третьих, нас не интересует информация, которая описывается после указания опыта работы (периоды работы в различных компаниях)
* В-четвертых, у нас есть проблема: опыт работы может быть представлен только в годах или только месяцах. Например, можно встретить следующие варианты:
    * Опыт работы 3 года 2 месяца…
    * Опыт работы 4 года…
    * Опыт работы 11 месяцев…
    * Учитывайте эту особенность в вашем коде

Учитывайте эту особенность в вашем коде

В результате преобразования у вас должен получиться столбец, содержащий информацию о том, сколько месяцев проработал соискатель.
Выполните преобразование, ответьте на контрольные вопросы и удалите столбец **"Опыт работы"** из таблицы.


In [ ]:
def get_experience(experience):
    """Функция для преобразования признака "Опыт работы",
    в новый признак "Опыт работы (в месяцах)". 
    Для пустых значений возвращвет NaN

    Args:
        experience: строка столбца "Опыт работы".

    Returns:
       experience_value: Опыт работы в месяцах
    """
    # возвращаем NaN, если пропуски или скрытй пропуск:
    if experience is np.nan or experience=='Не указано':
        return np.nan
    # проводим замену непустых значений
    else:
        # делим строку по словам
        experience_list=experience.split(' ')[:7]
        #  количесвто лет и месяцев опыта
        number_year=0
        number_month=0
        # ключи поиска
        month_word=['месяц', 'месяцев', 'месяца']
        year_word=['год', 'года', 'лет']
        # в цикле ищем  года и месеца опыта по ключам
        for index, item in enumerate(experience_list):
            if item in year_word:
                number_year=int(experience_list[index-1])
            if item in month_word:
                number_month=int(experience_list[index-1])
        
        experience_value=number_year*12 + number_month
        return experience_value
    
# применяем функцию к признаку 'Опыт работы'              
hh_df['Опыт работы (в меcяцах)']=hh_df['Опыт работы'].apply(get_experience)
hh_df=hh_df.drop('Опыт работы', axis=1)

# медианный опыт работы (в месяцах)
print(round(hh_df['Опыт работы (в меcяцах)'].median()))

# 100

### 4. Хорошо идем! Следующий на очереди признак "Город, переезд, командировки". Информация в нем представлена в следующем виде: **<Город , (метро) , готовность к переезду (города для переезда) , готовность к командировкам>**. В скобках указаны необязательные параметры строки. Например, можно встретить следующие варианты:

* Москва , не готов к переезду , готов к командировкам
* Москва , м. Беломорская , не готов к переезду, не готов к командировкам
* Воронеж , готов к переезду (Сочи, Москва, Санкт-Петербург) , готов к командировкам

Создадим отдельные признаки **"Город"**, **"Готовность к переезду"**, **"Готовность к командировкам"**. При этом важно учесть:

* Признак **"Город"** должен содержать только 4 категории: "Москва", "Санкт-Петербург" и "город-миллионник" (их список ниже), остальные обозначьте как "другие".

    Список городов-миллионников:
    
   <code>million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 'Челябинск','Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']
    </code>
    Инфорация о метро, рядом с которым проживает соискатель нас не интересует.
* Признак **"Готовность к переезду"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к переезду в признаке "Город, переезд, командировки". Например:
    * … , готов к переезду , …
    * … , не готова к переезду , …
    * … , готова к переезду (Москва, Санкт-Петербург, Ростов-на-Дону)
    * … , хочу переехать (США) , …
    
    Нас интересует только сам факт возможности или желания переезда.
* Признак **"Готовность к командировкам"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к командировкам в признаке "Город, переезд, командировки". Например:
    * … , готов к командировкам , … 
    * … , готова к редким командировкам , …
    * … , не готов к командировкам , …
    
    Нас интересует только сам факт готовности к командировке.
    
    Еще один важный факт: при выгрузки данных у некоторых соискателей "потерялась" информация о готовности к командировкам. Давайте по умолчанию будем считать, что такие соискатели не готовы к командировкам.
    
### Выполняем преобразования и удалите столбец **"Город, переезд, командировки"** из таблицы.

*Совет: обратите внимание на то, что структура текста может меняться в зависимости от указания ближайшего метро. Учите это, если будете использовать порядок слов в своей программе.*


In [ ]:
#Выделим признак "Город"
hh_df["Город"] = hh_df["Город, переезд, командировки"].apply(lambda x: x.split(' ,')[0]) 

def change_city(city):   #преобразуем значения городов до 4 признаков.
    million_cities = [
        'Новосибирск', 'Екатеринбург', 'Нижний Новгород',
        'Казань', 'Челябинск', 'Омск', 'Самара',
        'Ростов-на-Дону', 'Уфа', 'Красноярск',
        'Пермь', 'Воронеж', 'Волгоград'
    ]
    if (city == 'Москва') or (city == 'Санкт-Петербург'):
        return city
    elif city in million_cities:
        return 'город миллионник'
    else:
        return 'другие'
    
    
hh_df["Город"] = hh_df["Город"].apply(change_city)
hh_df["Город"].unique()

#Найдем процент соискателей в проживающих в Санкт-Петербурге
hh_df["Город"].value_counts(normalize=True).round(2)*100

In [ ]:
#Выделим признак "Готовность к переезду"
def relocation_transformer(v):
    if ('не готов к переезду' in v) or ('не готова к переезду' in v):
        return False
    else:
        return True

    
hh_df["Готовность к переезду"] = hh_df["Город, переезд, командировки"].apply(relocation_transformer)
hh_df["Готовность к переезду"].value_counts()

In [ ]:
#Выделим признак "Готовность к командировкам"
def business_trip_transformer(v):
    if ('не готов к командировкам' in v) or('не готова к командировкам' in v):
        return False
    else:
        return True


hh_df["Готовность к командировкам"] = hh_df["Город, переезд, командировки"].apply(business_trip_transformer)
hh_df["Готовность к командировкам"].value_counts()

In [ ]:
#удаляем столбец "Город, переезд, командировки"
hh_df = hh_df.drop('Город, переезд, командировки', axis=1)

#Найдем процент соискателей готовых одновременно и к переездам, и к командировкам
round(hh_df[hh_df['Готовность к переезду'] & hh_df['Готовность к командировкам']].shape[0] / hh_df.shape[0] *100)

5. Рассмотрим поближе признаки **"Занятость"** и **"График"**. Сейчас признаки представляют собой набор категорий желаемой занятости (полная занятость, частичная занятость, проектная работа, волонтерство, стажировка) и желаемого графика работы (полный день, сменный график, гибкий график, удаленная работа, вахтовый метод).
На сайте hh.ru соискатель может указывать различные комбинации данных категорий, например:
* полная занятость, частичная занятость
* частичная занятость, проектная работа, волонтерство
* полный день, удаленная работа
* вахтовый метод, гибкий график, удаленная работа, полная занятость

Такой вариант признаков имеет множество различных комбинаций, а значит множество уникальных значений, что мешает анализу. Нужно это исправить!

Давайте создадим признаки-мигалки для каждой категории: если категория присутствует в списке желаемых соискателем, то в столбце на месте строки рассматриваемого соискателя ставится True, иначе - False.

Такой метод преобразования категориальных признаков называется One Hot Encoding и его схема представлена на рисунке ниже:

Выполните данное преобразование для признаков "Занятость" и "График", ответьте на контрольные вопросы, после чего удалите их из таблицы

In [ ]:
mask = {
    'Занятость': ['полная занятость', 'частичная занятость', 'проектная работа', 'волонтерство', 'стажировка'],
    'График': ['полный день', 'сменный график', 'гибкий график', 'удаленная работа', 'вахтовый метод']
}
for key in mask:
    for i in mask[key]:
        hh_df[i] = hh_df[key].apply(lambda x: i in x)

#удаляем столбцы "Занятость" и "График"        
hh_df.drop(mask.keys(), axis=1, inplace=True)

#Найдем количество людей, которые ищут проектную работу или волонтёрство
a = hh_df[hh_df['проектная работа'] & hh_df['волонтерство']].shape[0]
display(f"количество людей, которые ищут проектную работу или волонтёрств: {a}")

#Найдем количество людей, которые хотят работать вахтовым методом или с гибким графиком
b = hh_df[hh_df['вахтовый метод'] & hh_df['гибкий график']].shape[0]
display(f"количество людей, которые хотят работать вахтовым методом или с гибким графикомв: {b}")

### 6. (2 балла) Наконец, мы добрались до самого главного и самого важного - признака заработной платы **"ЗП"**. 
В чем наша беда? В том, что помимо желаемой заработной платы соискатель указывает валюту, в которой он бы хотел ее получать, например:
* 30000 руб.
* 50000 грн.
* 550 USD

Нам бы хотелось видеть заработную плату в единой валюте, например, в рублях. Возникает вопрос, а где взять курс валют по отношению к рублю?

На самом деле язык Python имеет в арсенале огромное количество возможностей получения данной информации, от обращения к API Центробанка, до использования специальных библиотек, например pycbrf. Однако, это не тема нашего проекта.

Поэтому мы пойдем в лоб: обратимся к специальным интернет-ресурсам для получения данных о курсе в виде текстовых файлов. Например, MDF.RU, данный ресурс позволяет удобно экспортировать данные о курсах различных валют и акций за указанные периоды в виде csv файлов. Мы уже сделали выгрузку курсов валют, которые встречаются в наших данных за период с 29.12.2017 по 05.12.2019. Скачать ее вы можете **на платформе**

Создайте новый DataFrame из полученного файла. В полученной таблице нас будут интересовать столбцы:
* "currency" - наименование валюты в ISO кодировке,
* "date" - дата, 
* "proportion" - пропорция, 
* "close" - цена закрытия (последний зафиксированный курс валюты на указанный день).


Перед вами таблица соответствия наименований иностранных валют в наших данных и их общепринятых сокращений, которые представлены в нашем файле с курсами валют. Пропорция - это число, за сколько единиц валюты указан курс в таблице с курсами. Например, для казахстанского тенге курс на 20.08.2019 составляет 17.197 руб. за 100 тенге, тогда итоговый курс равен - 17.197 / 100 = 0.17197 руб за 1 тенге.
Воспользуйтесь этой информацией в ваших преобразованиях.




Осталось только понять, откуда брать дату, по которой определяется курс? А вот же она - в признаке **"Обновление резюме"**, в нем содержится дата и время, когда соискатель выложил текущий вариант своего резюме. Нас интересует только дата, по ней бы и будем сопоставлять курсы валют.

Теперь у нас есть вся необходимая информация для того, чтобы создать признак "ЗП (руб)" - заработная плата в рублях.

После ответа на контрольные вопросы удалите исходный столбец заработной платы "ЗП" и все промежуточные столбцы, если вы их создавали.

Итак, давайте обсудим возможный алгоритм преобразования: 
1. Перевести признак "Обновление резюме" из таблицы с резюме в формат datetime и достать из него дату. В тот же формат привести признак "date" из таблицы с валютами.
2. Выделить из столбца "ЗП" сумму желаемой заработной платы и наименование валюты, в которой она исчисляется. Наименование валюты перевести в стандарт ISO согласно с таблицей выше.
3. Присоединить к таблице с резюме таблицу с курсами по столбцам с датой и названием валюты (подумайте, какой тип объединения надо выбрать, чтобы в таблице с резюме сохранились данные о заработной плате, изначально представленной в рублях). Значение close для рубля заполнить единицей 1 (курс рубля самого к себе)
4. Умножить сумму желаемой заработной платы на присоединенный курс валюты (close) и разделить на пропорцию (обратите внимание на пропуски после объединения в этих столбцах), результат занести в новый столбец "ЗП (руб)".


In [ ]:
# создаем новый DataFrame с курсами валют
exchange_df=pd.read_csv('data/ExchangeRates.csv')
# расчет курса с учетом пропорции
exchange_df['rate']=exchange_df['close']/exchange_df['proportion']
# удаляем неинтересующие нас столбцы
exchange_df=exchange_df.drop(
    ['per', 'time', 'vol', 'close', 'proportion'], 
    axis=1
    )

# разделим признак "ЗП" на два признака "ЗП (значение)" и "ЗП (валюта)"
def get_salary_num(arg):
    """функция разделяет значение желаемой заработной 
    платы от наименования валюты

    Args:
        arg: запись в строке "ЗП"
        
    Returns:
         salary: значение желаемой ЗП
    """
    salary=float(arg.split()[0])
    return salary

def get_currency(arg):
    """функция разделяет наименования валюты 
     от размера заработной платы

    Args:
        arg: запись в строке "ЗП"
        
    Returns:
         валюта в ISO-кодировке
    """
    
    # словарь, в котором ключ - это наименование валюты в данных, 
    # а значение - наименование валюты в ISO-кодировке
    currency_dict={
        'грн':'UAH', 'USD':'USD',
        'EUR':'EUR', 'белруб':'BYN', 
        'KGS':'KGS','сум':'UZS',
        'AZN':'AZN','KZT':'KZT'
    }
    
    # заменяем наименование валюты в ISO-кодировку
    currency=arg.split()[1].replace('.','')
    if currency=='руб':
        return 'RUB'
    else:
        return currency_dict[currency]
    
#применяем функции к столбцу "ЗП:"
hh_df['ЗП (значение)']=hh_df['ЗП'].apply(get_salary_num)
hh_df['ЗП (валюта)']=hh_df['ЗП'].apply(get_currency
                                           )
# переводим признак 'Обновления резюме' основного датасета и 
# признак 'date' таблицы с курсами валют в формат datetime
hh_df['Обновление резюме']=pd.to_datetime(hh_df['Обновление резюме']).dt.date
exchange_df['date']=pd.to_datetime(exchange_df['date']).dt.date

#объединяем таблицы:
merged=hh_df.merge(
    exchange_df,
    left_on=['ЗП (валюта)', 'Обновление резюме'],
    right_on=['currency','date'],
    how='left'  
)
# заполняем пропуски в графе "rate" на 1:
merged['rate']=merged['rate'].fillna(1)

# создаем признак "ЗП (в рублях)":
hh_df['ЗП (в рублях)']=merged['ЗП (значение)']*merged['rate']
# удаляем столбцы "ЗП", "ЗП (значение)", "ЗП (валюта)":
hh_df=hh_df.drop(['ЗП','ЗП (значение)','ЗП (валюта)'], axis=1)

# Желаемая медианная заработная плата. В ответе указать целое число тысяч
print(round(hh_df['ЗП (в рублях)'].median()/1000))

# 59

# Исследование зависимостей в данных

### 1. Постройте распределение признака **"Возраст"**. Опишите распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится возраст большинства соискателей? Есть ли аномалии для признака возраста, какие значения вы бы причислили к их числу?
*Строим гистограмму и коробчатую диаграмму рядом.*

In [ ]:
fig = px.histogram(
    hh_df,
    x='Возраст',
    title='Распределение по возрасту',
    marginal='box'
)
fig.update_layout(yaxis_title="Общее количество")
fig.show()   

### Выводы по графику:
#### Анализ соискателей по возрасту:
* 1. Распределение по возрасту почти нормальное, так как мода смещена немного влево.
* 2. Мода распределения - 30 лет.
* 3. Предельные значения от 14 до 50 лет.
* 4. Большинство соискателей в возрасте примерно от 23 до 35 лет.
#### Анализ по возрасту совпадает с реальным ожиданием, но есть кое-какие аномалии:
* 1. Один соискатель с возрастом 100 лет, с не большим опытом.
* 2. Соискатель с возрастом 14 лет.
#### Требует дополнительной проверки записи соискателей возрастом старше 65 лет.
#### Такая картина распределения вполне объясняется поведением людей на рынке труда: по окончании учебных заведений (или уже во время учебы) человек ищет работу или же идет получать более высокое образование (количество соискателей растет), с увелечением возраста и получением опыта люди ищут работу "где лучше". Кроме того, люди средней возрастной категории (30-40 лет) зачастую находится в перманентном поиске работы, имея работу. С возрастом люди становятся менее готовыми к новой работе и число соискателей в этих возрастных группах умеьшается.¶

### 2. Построим распределение признака **"Опыт работы (месяц)"**. 
Опишем данное распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится опыт работы большинства соискателей? Есть ли аномалии для признака опыта работы, какие значения вы бы причислили к их числу?
#### *Строим гистограмму и коробчатую диаграмму рядом.*

In [ ]:
# гиcтограмма и коробчатая диаграмма распределения опыта работы соискателей
fig=px.histogram(
    hh_df,
    x='Опыт работы (в меcяцах)',
    title='Распределение соискателей по опыту',
    marginal='box'
)

fig.show()

#### Анализ соискателей по опыту:
1. Мода опыта работы - 81 месяц, медианное значение - 100 месяцев
2. Предельные значения - от 1 до 299 месяцев.
3. Большинство кандидатов имеют имеют опыт от 3 до 200 месяцев.
4. Анализ по опыту совпадает с реальным ожиданием, но есть аномалии:
* - Кандидат с опытом 99 лет (явный выброс)
#### Таким образом, можно сказать, что распределение соискателей по опыту работы, намоминает, характер логнормального.
#### Требуется дополнительная проверка записи соискателей с большим опытом работы

### 3. Построим распределение признака **"ЗП (руб)"**. Опишем данное распределение, отвечая на следующие вопросы: 
* - Каковы предельные значения признака, в каком примерном интервале находится заработная плата большинства соискателей? 
* - Есть ли аномалии для признака ЗП? 
* - Обратим внимание на гигантские размеры желаемой заработной платы.
#### *Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
#Найдем сколько соискателей требуют заработную плату выше 1 миллиона рублей
a = hh_df[hh_df['ЗП (в рублях)'] > 1000000]['ЗП (в рублях)'].count()
display(f'{a} соискателей требуют заработную плату выше 1 миллиона рублей')


fig = px.histogram(
    hh_df,
    x='ЗП (в рублях)',
    title='Распределение по размеру желаемой зарплаты (руб)',
    marginal='box'
)
fig.update_layout(yaxis_title="Общее количество")
fig.show()

### Анализ соискателей по требуемой зарплате:

* Мода желаемой зарплаты - 50 тысяч рублей.
* Предельные значения - от 17 500 до 24 300 000 рублей.
* Большинство соискателей расчитывают получать з.п. от 22 000 до 100 000.

#### Нужно сделать анализ по желаемой з.п. и провести очистку:
* Есть кандидаты с очень большим требованием по зарплате, возможно указана плата за год.
* Параметры зарплаты имеют очень большой разброс, требуется дополнительно отсеить данные.
#### Интервал желаемой заработной платы большинства соискателей - 37 тыс. -181 тыс. рублей; минимальная заработная плата - 1 рубль, максимальная заработная плата - 24 млн. рублей
#### К аномалиям предположительно можно отнести заработную плату больше 1 млн. руб (выбросы на графике), а также слишком низкую заработную плату (1,0 руб)¶
#### Разбег желаемой заработной платы очень велик. Требуют дополнительной проверки слишком низкие и слишком высокие желаемые заработные платы.
#### Такой разбег по желаемой заработной плате можно объяснить тем, что у соискателей с различным опытом, образованием, профессией, претендуемой должностью, условиями работы, разные и ожидания размера заработной платы.

### 4. Построим диаграмму, которая показывает зависимость **медианной** желаемой заработной платы (**"ЗП (руб)"**) от уровня образования (**"Образование"**). Используем для диаграммы данные о резюме, где желаемая заработная плата меньше 1 млн рублей.
### *Сделаем выводы по представленной диаграмме: для каких уровней образования наблюдаются наибольшие и наименьшие уровни желаемой заработной платы? Как вы считаете, важен ли признак уровня образования при прогнозировании заработной платы?*

In [ ]:
#Сгруппируем данные по зарплате (до 1 млн) и образованию, выведем медианную зарплату.
group1_df = hh_df[hh_df['ЗП (в рублях)']<1_000_000].groupby('Образование', as_index=False)['ЗП (в рублях)'].median()

fig = px.bar(
    group1_df,
    x='Образование',
    y='ЗП (в рублях)',
    title='Зависимость желаемой зарплаты от образования',
    width=900,
    height=500
)
fig.show()

In [ ]:
# фильтруем данные по заработной плате ниже 1 млн. руб.
filtered_by_salary=hh_df[hh_df['ЗП (в рублях)']<1e6]
# группируем данные по категориям образования и медианной ЗП
bar_grouped=filtered_by_salary.groupby(
    by='Образование', 
    as_index=False
)['ЗП (в рублях)'].median()

# строим столбчатую диаграмму зависимости медианной ЗП от образования
fig=px.bar(
    data_frame=bar_grouped,
    x='Образование',
    y='ЗП (в рублях)',
    color='Образование',
    title='Зависимость медианной желаемой заработной платы от уровня образования'
)

fig.show()

### Выводы:
* - Чем выше уровень образования, тем выше требуемая зарплата
* - Уровень образование является важным признаком для уровня требуемой зарплаты.

### 5. Построим диаграмму, которая показывает распределение желаемой заработной платы (**"ЗП (руб)"**) в зависимости от города (**"Город"**). Используем для диаграммы данные о резюме, где желая заработная плата меньше 1 млн рублей.
* *Сделаем выводы по полученной диаграмме: как соотносятся медианные уровни желаемой заработной платы и их размах в городах?* 
* *Можно ли считать, важен ли признак города при прогнозировании заработной платы?*

In [ ]:
# коробчатая диаграмма распределения желаемой ЗП по городам
fig=px.box(
    data_frame=filtered_by_salary,
    y='Город',
    x='ЗП (в рублях)',
    color='Город',
    title='Распределение желаемой заработной платы по городам'
)

fig.show()

### Согласно графикам распределния желаемой заработной платы по городам, можно отметить:
1. Медианная заработная плата варьируется от 40 тыс. руб в городах-миллиониках и городах категории "другии" до 85 тыс. руб в Москве, в Санкт-Петербурге медианная заработная плата несколько выше, чем в городах-миллиониках - 44 тыс. рублей.
2.  Основная масса соискателей в городах-миллиониках и других городах имеет размах ЗП: от 1 до 105 тыс. руб, в Санкт-Петербурге - от 1 до 145 тыс. рублей, в Москве - от 1 до 285 тыс. рублей.¶
#### Т.е. желаемые заработные платы в Москве примерно в 2 раза выше, чем по стране; зарплаты в Санкт-Петербурге несколько выше, чем в целом по стране. Различий по заработной плате между городами-миллиониками и городами категории "другие" нет
#### Таким образом, признак "Город" важен при прогнозировании заработной платы, однако города-миллионики и города "другие" можно объединить в одну категорию. Такие различия по желаемой заработной плате можно объяснить сложившейся по ряду причин ситуацией на рынке труда в стране - в Москве уровень зароботных плат выше, чем в регионах

### 6. Построим **многоуровневую столбчатую диаграмму**, которая показывает зависимость медианной заработной платы (**"ЗП (руб)"**) от признаков **"Готовность к переезду"** и **"Готовность к командировкам"**. Проанализируем график, сравнив уровень заработной платы в категориях.

In [ ]:
# группируем данные по готовности к перезду/командировкам и медианной заработной плате
bar_grouped=hh_df.groupby(['Готовность к командировкам', 'Готовность к переезду'], as_index=False)['ЗП (в рублях)'].median()
# создаем колонки, в которых информация о готовности записана текстом:
bar_grouped['text_move']=bar_grouped['Готовность к переезду'].apply(lambda x: 'готов к переезду' if x==True  else 'не готов к переезду')
bar_grouped['text_trip']=bar_grouped['Готовность к командировкам'].apply(lambda x: 'готов к командировкам' if x==True  else 'не готов к командировкам')

# строим многоуровневую столбчатую диаграмму зависимости ЗП 
# от готовности к командировкам/переезду
fig=px.bar(
    data_frame=bar_grouped,
    x='text_move',
    y='ЗП (в рублях)',
    barmode='group',
    color='text_trip',
    title='Медианная заработная плата по готовности к командировкам/переезду'
    )

fig.show()

In [ ]:
group2_df = hh_df.groupby(
    ['Готовность к переезду', 'Готовность к командировкам'],
    as_index=False)['ЗП (в рублях)'].median()

fig = px.bar(
    group2_df,
    y='Готовность к командировкам', 
    x='ЗП (в рублях)',
    barmode="group",
    color='Готовность к переезду',
    orientation='h',
    title='Медианная з/п по готовности к командировкам/переезду'
)

fig.show()

### Согласно диаграмме можно отметить следующее:
* - Наименьшая желаемая медианная зароботная плата (45 тыс. руб) у соискателей, которые не готовы ни к переезду, ни к командировкам.
* - Наибольшая - у соискателей готовых к переезду и командировкам - 66 тыс. руб.
* - Медианная заработная плата соискателей готовых к переезду выше по отношению к соискателем, нежелающим переезжать.
* - Аналогичная зависимость и у соискателей по отношению к командировкам - у соискателей, готовых к командировкам зароботная плата выше, чем у неготовых.¶
#### Таким образом, желаемая заработная плата выше у соискателей готовых к командировкам и к переезду.

### 7. Построим сводную таблицу, иллюстрирующую зависимость **медианной** желаемой заработной платы от возраста (**"Возраст"**) и образования (**"Образование"**). На полученной сводной таблице постройте **тепловую карту**. Проанализируем тепловую карту, сравнив показатели внутри групп.

In [ ]:
# строим сводную таблицу зависимости медианной ЗП от образования и возраста
pivot=hh_df.pivot_table(
    values='ЗП (в рублях)',
    index='Образование',
    columns='Возраст',
    aggfunc='median',
    fill_value=0
)

fig=px.imshow(
    pivot, 
    aspect='auto',
    title='Тепловая карта мединной заработной платы от возраста и образования'
)

fig.show()

In [ ]:
pivot1_df = hh_df.pivot_table(
    index='Возраст',
    columns='Образование',
    values='ЗП (в рублях)',
    aggfunc='median',
)

fig = px.imshow(
    pivot1_df,
    color_continuous_scale='dense',
    title='Тепловая карта мединной заработной платы от возраста и образования'
)
fig.update_yaxes(autorange=True)

fig.show()

In [ ]:
pivot = hh_df.pivot_table(
    index='Образование',
    columns='Возраст',
    values='ЗП (в рублях)',
    aggfunc='median',
    fill_value=0
)
fig = px.imshow(
    pivot,
    aspect='auto',
    color_continuous_scale='greens',
    title='Медианная з/п по образованию и возрасту'
)
fig.show()

### Bыводы: 
* - Разброс медианных желаемых зарплат по возрастам достаточно велик от практически 0 до 120 тыс. руб.
* - В общем с ростом возраста желаемая заработная плата растет.¶
* - Интенсивность роста желаемой заработной платы с возрастом выше у соискателей с высшим образованием.
* - Наименьшая интенсивность роста заработной платы у соискателей со средним специальным образованием.
#### Таким образом - признак "Образование"  важен для прогнозирования зароботной платы
#### Чем больше возраст, тем больше опыт, тем больше развиты компетенции, тем выше ожидания по заработной плате.

### 8. Построим **диаграмму рассеяния**, показывающую зависимость опыта работы (**"Опыт работы (месяц)"**) от возраста (**"Возраст"**). Опыт работы переведём из месяцев в года, чтобы признаки были в едином масштабе. Построим на графике дополнительно прямую, проходящую через точки (0, 0) и (100, 100). Данная прямая соответствует значениям, когда опыт работы равен возрасту человека. Точки, лежащие на этой прямой и выше нее - аномалии в наших данных (опыт работы больше либо равен возрасту соискателя)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig = plt.figure(figsize=(10, 5))
scatter_data = hh_df.copy()
scatter_data['Опыт работы (год)'] = scatter_data['Опыт работы (в меcяцах)']/12
sns.lineplot(x=[0, 100], y=[0, 100])
ax = sns.scatterplot(
    data=scatter_data, 
    x='Возраст',
    y='Опыт работы (год)',
)
ax.set_title('Зависимость опыта работы от возраста');

In [ ]:
# переводим опыт работы из месяцев в года
hh_df['Опыт работы (в годах)']=hh_df['Опыт работы (в меcяцах)']/12
# диаграмма рассеяния опыта работы (в годах) от возраста
fig1=px.scatter(
    data_frame=hh_df,
    x='Возраст',
    y='Опыт работы (в годах)',
    title='Зависимость опыта работы от возраста',
)

# строим прямую линию:
data_line=pd.DataFrame({'x':[0,100],'y':[0,100]}) #DataFrame для графика
fig2=px.line(data_line, x='x', y='y')

# объединяем графики и выводим на экран
fig=go.Figure(data=fig1.data+fig2.data)
fig.show()

### Как видно на графике, у соискателей в молодом возрасте опыт небольшой или отсутствует. С увелечением возраста увеличивается разброс по опыту. Семь значений лежит выше прямой x=y, т.е. опыт работы превышает возраст человека, что невозможно - эти значения являются аномалиями.

## **Дополнительные баллы**
#### Для получения 2 дополнительных баллов по разведывательному анализу постройте еще два любых содержательных графика или диаграммы, которые помогут проиллюстрировать влияние признаков/взаимосвязь между признаками/распределения признаков. Приведите выводы по ним. Желательно, чтобы в анализе участвовали признаки, которые мы создавали ранее в разделе "Преобразование данных".
### Построим две столбчатые диаграммы зависимости медианной заработной платы от 
#### 1). типа занятости; 
#### 2). графика работы для определения влияния этих признаков на размер заработной платы

In [ ]:
# список столбцов с типами занятости и графиками работы
cols=['полная занятость',
      'частичная занятость',
      'проектная работа',
      'волонтерство', 
      'стажировка',
      'полный день',
      'сменный график',
      'гибкий график',
      'удаленная работа', 
      'вахтовый метод'
]
# проходим в цикле по колонкам  и группируем данные
df_cols=pd.DataFrame() # создаем пустую таблицу
for col in cols:
  grouped_col=hh_df[hh_df[col]==True].groupby(
       by=col, 
       as_index=False
       )['ЗП (в рублях)'].median()
  # объединяем, сгруппированные данные
  df_cols=pd.concat(
      [df_cols, grouped_col], 
      join='outer',
      ignore_index=True
  )

# cтроим столбчатую диаграмму зависимости медианной ЗП от занятости
fig=px.bar(
    data_frame=df_cols,
    y='ЗП (в рублях)',
    title='Медианная заработная плата по виду занятости и графику работы',
    color=cols
)

fig.show()

In [ ]:
#Найдем соотношение по полу соискателя
gender_df = hh_df.groupby(by="Пол", as_index=True)["Пол"].count()
    
fig = px.pie(
    gender_df, 
    values="Пол",
    names=gender_df.index.tolist(),
    hole=.25, 
    color_discrete_sequence=["green", "red"],
    height=500,
    width=900,
    title='Соотношение мужчин и женщин среди соискателей'
)
fig.update_traces(
    textposition='inside',
    textinfo='percent+label'
)
 
fig.show()

### Мужчин среди соискателей гораздо больше. Причин может быть море. Просто обратим внимание на огромную диспропорцию.

In [ ]:
#Найдем соотношение по желаемой зарплате по полу соискателя
fig = px.box(
    hh_df, 
    y='Пол', 
    x='ЗП (в рублях)', 
    title='Распределение желаемой зарплате по полу соискателя', 
    color='Пол',
    height=500,
    width=900
)

#Подправим шкалу, так как выбросы сильно искривляют график
fig.update_layout(xaxis_range=[0,200000], showlegend=False)

### Женщины готовы пойти на меньшую зарплату. 

In [ ]:
#Посмотрим на распределение по образованию у мужчин и женщин
px.histogram(
    hh_df, 
    x='Образование', 
    title='Распределение по образованию у мужчин и женщин', 
    color='Пол',
    height=500,
    width=900
)

### Женщины преобладающе имеют высшее образование, а техникумы и cреднее  в основном мужчины.

# Очистка данных

### 1. Начнем с дубликатов в наших данных. Найдите **полные дубликаты** в таблице с резюме и удалите их. 

In [ ]:
#Перед очисткой сделаем копию таблицы
new_hh_df = hh_df.copy()
new_hh_df.head(2)

In [ ]:
duplicates = new_hh_df[new_hh_df.duplicated(subset=new_hh_df.columns)]
new_hh_df = new_hh_df.drop_duplicates()
print(duplicates.shape[0])

### 2. Займемся пропусками. Выведите информацию **о числе пропусков** в столбцах. 

In [ ]:
null_df = new_hh_df.isnull().sum()
display(null_df[null_df > 0])

### 3. Итак, у нас есть пропуски в 3ех столбцах: **"Опыт работы (месяц)"**, **"Последнее/нынешнее место работы"**, **"Последняя/нынешняя должность"**. Поступим следующим образом: удалите строки, где есть пропуск в столбцах с местом работы и должностью. Пропуски в столбце с опытом работы заполните **медианным** значением.

In [ ]:
#  удаляем
new_hh_df = new_hh_df.dropna(subset=['Последнее/нынешнее место работы', 'Последняя/нынешняя должность'])

# заполним пропуски в столбце "Опыт работы (в месяцах)" медианным значением
new_hh_df['Опыт работы (в меcяцах)'] = new_hh_df['Опыт работы (в меcяцах)'].fillna(new_hh_df['Опыт работы (в меcяцах)'].median())
print(round(new_hh_df['Опыт работы (в меcяцах)'].mean()))

### 4. Мы добрались до ликвидации выбросов. Сначала очистим данные вручную. Удалите резюме, в которых указана заработная плата либо выше 1 млн. рублей, либо ниже 1 тыс. рублей.

In [ ]:
outliers = new_hh_df[(new_hh_df['ЗП (в рублях)'] > 1e6) | (new_hh_df['ЗП (в рублях)'] < 1e3)]
new_hh_df = new_hh_df.drop(outliers.index)
print(outliers.shape[0])

### 5. В процессе разведывательного анализа мы обнаружили резюме, в которых **опыт работы в годах превышал возраст соискателя**. Найдите такие резюме и удалите их из данных


In [ ]:
#Выделим данные по условиям выброса
outliers = new_hh_df[new_hh_df['Опыт работы (в меcяцах)']/12 >= new_hh_df['Возраст']]

#Удалим строки из данных
new_hh_df = new_hh_df.drop(outliers.index)
#Выведем количество удаленных выбросов
print(outliers.shape[0])

6. В результате анализа мы обнаружили потенциальные выбросы в признаке **"Возраст"**. Это оказались резюме людей чересчур преклонного возраста для поиска работы. Попробуйте построить распределение признака в **логарифмическом масштабе**. Добавьте к графику линии, отображающие **среднее и границы интервала метода трех сигм**. Напомним, сделать это можно с помощью метода axvline. Например, для построение линии среднего будет иметь вид:

`histplot.axvline(log_age.mean(), color='k', lw=2)`

В какую сторону асимметрично логарифмическое распределение? Напишите об этом в комментарии к графику.
Найдите выбросы с помощью метода z-отклонения и удалите их из данных, используйте логарифмический масштаб. Давайте сделаем послабление на **1 сигму** (возьмите 4 сигмы) в **правую сторону**.

Выведите таблицу с полученными выбросами и оцените, с каким возрастом соискатели попадают под категорию выбросов?

In [ ]:
#Возьмем функцию для реализации алгоритма метода z-отклонений
def outliers_z_score_mod(data, feature, log_scale=False, left=3, right=3):
    if log_scale:
        z = np.log(data[feature])
    else:
        z = data[feature]
    mu = z.mean()
    sigma = z.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(z < lower_bound) | (z > upper_bound)]
    cleaned = data[(z > lower_bound) & (z < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_z_score_mod(new_hh_df, 'Возраст', log_scale=True, right=4)
display(f'outliers: {outliers.shape[0]}')
display(f'cleaned: {cleaned.shape[0]}')

#display(outliers['Возраст'].unique())
#Возраст соискателей из выброса: 15 и 100 лет!!!


#Построем сравнительные графики распределения по возрасту до и после применения алгоритма метода z-отклонений
fig1 = px.histogram(
    new_hh_df['Возраст']
)

log = np.log(new_hh_df['Возраст']+1)
fig2 = px.histogram(
    log,
    nbins = 10000
)

#Объединим их для наглядности
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Нормальное распределение', 'Логарифмическое распределение')
)
fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig2['data'][0], row=1, col=2)

fig.update_xaxes(title_text='Возраст (норма)', row=1, col=1)
fig.update_xaxes(title_text='Возраст (логарифм)', row=1, col=2)

fig.update_yaxes(title_text="Общее количество", row=1, col=1)
fig.update_yaxes(title_text="Общее количество", row=1, col=2)

fig.update_layout(
    height=500, width=950,
    title_text="Распределение по возрасту",
    showlegend=False
)
fig.show()

### Выводы:
#### 1. Логарифмическое распределение имеет  отклонение влево.
#### 2. Возраст 15 лет и 100 лет попали в выбросы.